# Prueba de desarrollo

## Preparacion datasets

In [204]:
import pandas as pd
import numpy as np
import surprise as sp 

In [205]:
df = pd.read_parquet('../Datasets/parquet/Recomendacion/recomendacion.parquet')

df

,user_id,item_id,puntaje,item_name
0,76561197970982479,1250,4,Killing Floor
1,76561197970982479,22200,6,Zeno Clash
2,76561197970982479,43110,4,Metro 2033
3,js41637,251610,4,Barbie™ Dreamhouse Party™
4,js41637,227300,4,Euro Truck Simulator 2
...,...,...,...,...
51986,76561198312638244,130,4,Half-Life: Blue Shift
51987,76561198312638244,70,6,Half-Life
51988,76561198312638244,362890,4,Black Mesa
51989,LydiaMorley,273110,4,Counter-Strike Nexon: Zombies


In [206]:
df_ratings = df[['item_id','puntaje','user_id']]
df_ratings

,item_id,puntaje,user_id
0,1250,4,76561197970982479
1,22200,6,76561197970982479
2,43110,4,76561197970982479
3,251610,4,js41637
4,227300,4,js41637
...,...,...,...
51986,130,4,76561198312638244
51987,70,6,76561198312638244
51988,362890,4,76561198312638244
51989,273110,4,LydiaMorley


In [207]:
df_ratings.columns = ['itemID','rating','userID']
df_ratings

,itemID,rating,userID
0,1250,4,76561197970982479
1,22200,6,76561197970982479
2,43110,4,76561197970982479
3,251610,4,js41637
4,227300,4,js41637
...,...,...,...
51986,130,4,76561198312638244
51987,70,6,76561198312638244
51988,362890,4,76561198312638244
51989,273110,4,LydiaMorley


In [208]:
reader = sp.Reader(rating_scale=(1,6))

In [209]:
data = sp.Dataset.load_from_df(df_ratings[["userID", "itemID", "rating"]], reader)

In [210]:
trainset, _ = sp.model_selection.train_test_split(data, test_size=.000000001,random_state=43)
trainset2, _ = sp.model_selection.train_test_split(data, test_size=.000000001,random_state=10)

## Desarrollo recomendacion_usuario

In [211]:
model = sp.SVD()

In [212]:
model.fit(trainset)

In [214]:
df_titles = pd.read_parquet('../Datasets/parquet/Recomendacion/Final/diccionario_juegos.parquet')

In [215]:
df_titles.columns = ['itemID','itemName']

In [216]:
usuario = '00True'
usuario_vistas = df_ratings[df_ratings['userID'] == usuario]
usuario_vistas

,itemID,rating,userID
34302,730,4,00True
34303,221100,6,00True


In [217]:
usuario_vistas['itemID'] = usuario_vistas.itemID.apply(int)

C:\Users\pedro\AppData\Local\Temp\ipykernel_23220\944485059.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usuario_vistas['itemID'] = usuario_vistas.itemID.apply(int)


In [219]:
recomendaciones_usuario = df_titles.copy()

recomendaciones_usuario = recomendaciones_usuario[~recomendaciones_usuario.itemID.isin(usuario_vistas.itemID.values)]
recomendaciones_usuario = recomendaciones_usuario.reset_index(drop=True)
recomendaciones_usuario.head()

,itemID,itemName
0,1250,Killing Floor
1,22200,Zeno Clash
2,43110,Metro 2033
3,251610,Barbie™ Dreamhouse Party™
4,227300,Euro Truck Simulator 2


In [220]:
recomendaciones_usuario['Estimate_byuser_Score'] = recomendaciones_usuario['itemID'].apply(lambda x: model.predict(usuario, x).est)

In [221]:
recomendaciones_usuario = recomendaciones_usuario.sort_values(f'Estimate_byuser_Score', ascending=False)
print(recomendaciones_usuario.head(10))

      itemID                            itemName  Estimate_byuser_Score
0       1250                       Killing Floor               4.461917
2217  320040                        Moon Hunters               4.461917
2207    6210                  Vegas: Make It Big               4.461917
2208  317040             Strife: Veteran Edition               4.461917
2209    3900         Sid Meier's Civilization IV               4.461917
2210  373360  Cast of the Seven Godsends - Redux               4.461917
2211  237590                  R.I.P.D.: The Game               4.461917
2212  209520               Mini Motor Racing EVO               4.461917
2213  423840                           BanHammer               4.461917
2214  444190         Princess Edge - Dragonstone               4.461917


In [223]:
recomendaciones_usuario[recomendaciones_usuario['itemID'].isin([4000,72850,9880,249130,105600])]

,itemID,itemName,Estimate_byuser_Score
2119,9880,Champions Online,4.461917
28,4000,Garry's Mod,4.461917
25,72850,The Elder Scrolls V: Skyrim,4.461917
21,249130,LEGO® MARVEL Super Heroes,4.461917
58,105600,Terraria,4.461917


## Desarrollo Recomendacion item

In [237]:
item = 1250
sim_options = {
    'name': 'cosine',
    'user_based': False  # Indica que es basado en ítems
}

model_item_based = sp.KNNBasic(sim_options=sim_options)
model_item_based.fit(trainset)


Computing the cosine similarity matrix...
Done computing similarity matrix.


In [277]:
def get_item_neighbors(item):
    try:
        iid = trainset.to_inner_iid(str(item))
        similar_items = model_item_based.get_neighbors(iid, k=5)
        result = []
        for i in similar_items:
            rid = trainset.to_raw_iid(i)
            rname = df_titles[df_titles.itemID == int(rid)].itemID.values
            result.append(rname[0])
        return result
    except:
        iid = trainset2.to_inner_iid(str(item))
        similar_items = model_item_based.get_neighbors(iid, k=5)
        result = []
        for i in similar_items:
            rid = trainset2.to_raw_iid(i)
            rname = df_titles[df_titles.itemID == int(rid)].itemID.values
            result.append(rname[0])
        return result

In [278]:
# Obtener ítems similares (top 5)
recomendaciones_usuario['itemRecomendation'] = recomendaciones_usuario.itemID.apply(get_item_neighbors)

#print(df_titles[df_titles.itemID.isin(similar_items)])

In [279]:
recomendaciones_usuario[recomendaciones_usuario.itemRecomendation == 'Error']

,itemID,itemName,Estimate_byuser_Score,itemRecomendation


In [280]:
recomendaciones_usuario

,itemID,itemName,Estimate_byuser_Score,itemRecomendation
0,1250,Killing Floor,4.461917,"[8870, 31220, 417860, 238460, 253430]"
2217,320040,Moon Hunters,4.461917,"[24240, 211820, 620, 377160, 65800]"
2207,6210,Vegas: Make It Big,4.461917,"[289130, 271590, 222880, 99900, 730]"
2208,317040,Strife: Veteran Edition,4.461917,"[218620, 8930, 374570, 233130, 365660]"
2209,3900,Sid Meier's Civilization IV,4.461917,"[105600, 208090, 221640, 234190, 212680]"
...,...,...,...,...
1109,229580,Dream,4.461917,"[730, 218620, 49520, 238320, 72200]"
1110,329830,The Moon Sliver,4.461917,"[238320, 72200, 229580, 242760, 305620]"
1111,413420,Danganronpa 2: Goodbye Despair,4.461917,"[730, 250320, 48700, 292030, 289130]"
1112,34030,Napoleon: Total War,4.461917,"[271590, 200210, 24240, 221100, 304930]"


In [262]:
recomendaciones_usuario[recomendaciones_usuario.itemID == 1250].itemRecomendation.values[0]

['BioShock Infinite',
 'Sam & Max 301: The Penal Zone',
 'Emily is Away',
 'BattleBlock Theater',
 'CastleMiner Z']

# Condensacion del brainstorm

## Recomendacion item

In [284]:
df_ratings = pd.read_parquet('../Datasets/parquet/Recomendacion/recomendacion.parquet')
df_titles = pd.read_parquet('../Datasets/parquet/Recomendacion/Final/diccionario_juegos.parquet')

df_ratings = df_ratings[['item_id','user_id','puntaje']]
df_ratings.columns = ['itemID','userID','rating']
df_titles.columns = ['itemID','itemName']

reader = sp.Reader(rating_scale=(1,6))
data = sp.Dataset.load_from_df(df_ratings[["userID", "itemID", "rating"]], reader)

trainset, _ = sp.model_selection.train_test_split(data, test_size=.000000001,random_state=43)
trainset2, _ = sp.model_selection.train_test_split(data, test_size=.000000001,random_state=10)

In [285]:
def get_item_neighbors(item):
    try:
        iid = trainset.to_inner_iid(str(item))
        similar_items = model_item_based.get_neighbors(iid, k=5)
        result = []
        for i in similar_items:
            rid = trainset.to_raw_iid(i)
            rname = df_titles[df_titles.itemID == int(rid)].itemID.values
            result.append(rname[0])
        return result
    except:
        iid = trainset2.to_inner_iid(str(item))
        similar_items = model_item_based.get_neighbors(iid, k=5)
        result = []
        for i in similar_items:
            rid = trainset2.to_raw_iid(i)
            rname = df_titles[df_titles.itemID == int(rid)].itemID.values
            result.append(rname[0])
        return result

In [286]:
df_titles['ItemRecomendations'] = df_titles.itemID.apply(get_item_neighbors)
df_titles

,itemID,itemName,ItemRecomendations
0,1250,Killing Floor,"[8870, 31220, 417860, 238460, 253430]"
1,22200,Zeno Clash,"[730, 8870, 57690, 1250, 49520]"
2,43110,Metro 2033,"[271590, 222880, 99900, 31220, 417860]"
3,251610,Barbie™ Dreamhouse Party™,"[200210, 239030, 227300, 289130, 271590]"
4,227300,Euro Truck Simulator 2,"[99900, 417860, 252490, 24240, 57690]"
...,...,...,...
3316,307130,Asteria,"[289130, 271590, 222880, 99900, 730]"
3317,209120,Street Fighter X Tekken,"[289130, 271590, 222880, 99900, 730]"
3318,220090,The Journey Down: Chapter One,"[265630, 262850, 289130, 271590, 222880]"
3319,262850,The Journey Down: Chapter Two,"[265630, 220090, 289130, 271590, 222880]"


In [293]:
# entonces veamos la formula completa para ambas funciones:

def recomendacion_juego(item_id):
    result = df_titles[df_titles['itemID'] == item_id].ItemRecomendations.values[0]
    resultado = []
    count = 1
    for i in result:
        resultado.append({'Puesto':count,'ID':i,'Nombre':df_titles[df_titles['itemID'] == i].itemName.values[0]})
        count +=1
    return resultado

recomendacion_juego(1250)

[{'Puesto': 1, 'ID': 8870, 'Nombre': 'BioShock Infinite'},
 {'Puesto': 2, 'ID': 31220, 'Nombre': 'Sam & Max 301: The Penal Zone'},
 {'Puesto': 3, 'ID': 417860, 'Nombre': 'Emily is Away'},
 {'Puesto': 4, 'ID': 238460, 'Nombre': 'BattleBlock Theater'},
 {'Puesto': 5, 'ID': 253430, 'Nombre': 'CastleMiner Z'}]

In [313]:
df_titles.to_parquet('../Datasets/parquet/Recomendacion/Surprise/df_titles.parquet')

## Recomendacion usuario


In [307]:
model = sp.SVD()
model.fit(trainset)

def recomendacion_masiva_usuario(usuario):

    usuario_vistas = df_ratings[df_ratings['userID'] == usuario]
    usuario_vistas.loc[:, 'itemID'] = usuario_vistas['itemID'].astype(int)

    recomendaciones_usuario = df_titles.copy()
    recomendaciones_usuario = recomendaciones_usuario[~recomendaciones_usuario.itemID.isin(usuario_vistas.itemID.values)]
    recomendaciones_usuario = recomendaciones_usuario.reset_index(drop=True)

    recomendaciones_usuario['Estimate_byuser_Score'] = recomendaciones_usuario['itemID'].apply(lambda x: model.predict(usuario, x).est)
    top5 = recomendaciones_usuario.sort_values(f'Estimate_byuser_Score', ascending=False).itemID[:5].values
    return list(top5)

recomendacion_masiva_usuario('00True')

[1250, 320040, 6210, 317040, 3900]

In [308]:
df_usuarios = pd.DataFrame(df_ratings.userID.drop_duplicates())
df_usuarios

,userID
0,76561197970982479
3,js41637
6,evcentric
12,doctr
18,maplemage
...,...
51982,76561198296402247
51983,76561198306599751
51984,Ghoustik
51985,76561198312638244


In [309]:
df_usuarios['userRecomendation'] = df_usuarios.userID.apply(recomendacion_masiva_usuario)

In [310]:
df_usuarios

,userID,userRecomendation
0,76561197970982479,"[251610, 320040, 6210, 317040, 3900]"
3,js41637,"[1250, 320040, 6210, 317040, 3900]"
6,evcentric,"[1250, 214870, 317040, 3900, 373360]"
12,doctr,"[1250, 214870, 317040, 3900, 373360]"
18,maplemage,"[1250, 214870, 317040, 3900, 373360]"
...,...,...
51982,76561198296402247,"[1250, 39690, 434420, 6210, 317040]"
51983,76561198306599751,"[1250, 39690, 434420, 6210, 317040]"
51984,Ghoustik,"[1250, 39690, 434420, 6210, 317040]"
51985,76561198312638244,"[1250, 214870, 317040, 3900, 373360]"


In [312]:
def recomendacion_usuario(user_id):
    result = df_usuarios[df_usuarios['userID'] == user_id].userRecomendation.values[0]
    resultado = []
    count = 1
    for i in result:
        resultado.append({'Puesto':count,'ID':i,'Nombre':df_titles[df_titles['itemID'] == i].itemName.values[0]})
        count +=1
    return resultado

recomendacion_usuario('00True')

[{'Puesto': 1, 'ID': 1250, 'Nombre': 'Killing Floor'},
 {'Puesto': 2, 'ID': 320040, 'Nombre': 'Moon Hunters'},
 {'Puesto': 3, 'ID': 6210, 'Nombre': 'Vegas: Make It Big'},
 {'Puesto': 4, 'ID': 317040, 'Nombre': 'Strife: Veteran Edition'},
 {'Puesto': 5, 'ID': 3900, 'Nombre': "Sid Meier's Civilization IV"}]

In [314]:
df_usuarios.to_parquet('../Datasets/parquet/Recomendacion/Surprise/df_usuarios.parquet')